In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize 
from deep_translator import GoogleTranslator
import pickle

nltk.download('stopwords')
nltk.download('punkt')

def text_preprocessing(text):
    stop_words = set(stopwords.words('english')) 
    stemmer = PorterStemmer(PorterStemmer.ORIGINAL_ALGORITHM)
  
    tokens = word_tokenize(text)
    splitted_words_without_stops = [word for word in tokens if not word in stop_words]
    splitted_words_without_puncs = [word for word in splitted_words_without_stops if word.isalnum()] 
    stemmed_words = [stemmer.stem(word) for word in splitted_words_without_puncs]

    return ' '.join(stemmed_words)

# Carregar o modelo LightGBM treinado
with open('../light_gbm/lightgbm_model.pkl', 'rb') as f:
    lgbm_model = pickle.load(f)

# Carregar o TFIDFVectorizer
with open('../light_gbm/tfidf_vectorizer_lgb.pkl', 'rb') as f:
    tfidf_vect = pickle.load(f)

# Função para traduzir o texto para inglês
def translate_to_english(text):
    translator = GoogleTranslator(source='auto', target='en')
    return translator.translate(text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Igor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Igor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def predict_sentiment(text):
    # Traduzir a frase para o inglês
    translated_text = translate_to_english(text)
    print(translated_text)
    # Preprocessar o texto traduzido
    preprocessed_text = text_preprocessing(translated_text)
    # Transformar o texto usando TFIDFVectorizer
    text_vector = tfidf_vect.transform([preprocessed_text])
    # Prever o sentimento usando o modelo treinado
    sentiment = lgbm_model.predict(text_vector)[0]
    if sentiment == 2:
        return 'Positive'
    elif sentiment == 1:
        return 'Neutral'
    else:
        return 'Negative'

In [17]:
# Exemplo de uso
input_text = "o quarto estava barulhento"
sentiment = predict_sentiment(input_text)
print(f"O sentimento da frase é: {sentiment}")

the room was noisy
O sentimento da frase é: Negative
